In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端机器学习：MLOps阶段7：监控：用于设置表格模型的Vertex AI模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_setup.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_setup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何设置用于表格模型的Vertex AI模型监测。

### 目标

在这个笔记本中，您将学习如何设置`Vertex AI模型监控`服务，以检测输入预测请求中的特征偏差和漂移。

本教程使用以下Google Cloud ML服务：

- `Vertex AI模型监控`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 下载一个预训练的自定义表格模型。
- 将预训练模型上传为`模型`资源。
- 将`模型`资源部署到`端点`资源。
- 配置`端点`资源以进行模型监控。
    - 用于特征输入的偏差和漂移检测。
    - 用于特征归因的偏差和漂移检测。
- 通过发送1000个预测请求自动生成`输入模式`。
- 列出、暂停、恢复和删除监控作业。
- 使用预定义的`输入模式`重新启动监控作业。
- 查看记录的监控数据。

### 模型

本教程使用一个预训练模型，模型工件存储在一个公共云存储桶中。

该模型基于[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。该模型的理念是，您的公司有大量描述游戏用户如何与网站互动的日志数据。原始数据包含以下类别的信息：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 关于玩家的信息，例如玩家所在的地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，如达到新的等级
- 流失倾向 - 这是标签或目标特征，它提供了这个玩家可能流失的估计概率，即停止成为活跃玩家。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解[Vertext AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

**否则**，请确保您的环境满足本笔记本的要求。您需要以下内容：

- Google Cloud SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南提供了[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)的详细说明，以满足这些要求。以下步骤提供了简化的一组说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

1. 要安装Jupyter，请在终端窗口中运行`pip install jupyter`命令。

1. 要启动Jupyter，请在终端窗口中运行`jupyter notebook`命令。

1. 在Jupyter Notebook仪表板中打开此笔记本。

安装
安装执行此笔记本所需的软件包。

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform \
                                             google-cloud-bigquery \
                                             tensorflow==2.7

### 重新启动内核

在安装完 SDK 后，您需要重新启动笔记本内核，以便它可以找到这些软件包。您可以从 *Kernel -> Restart Kernel* 重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您在什么笔记本环境中，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得$300的免费信用用于计算/存储费用。

1. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您是在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入到这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量在本笔记本的其余部分中用于操作。 以下是 Vertex AI 支持的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

**对于本笔记本，我们建议您将区域设置为默认值 `us-central1`**。

您不能在 Vertex AI 中使用多区域存储桶进行训练。并非所有区域都支持所有 Vertex AI 服务。

了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您在一个实时教程会话中，可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加在本教程中创建的资源名称后面。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

用户电子邮件

将您的用户电子邮件地址设置为接收监控警报。

In [ ]:
USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

### 验证您的谷歌云帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证。

**否则**，请按照以下步骤操作：

1. 在 Cloud 控制台中，前往【创建服务帐号密钥】页面(https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，然后选择**Vertex AI管理员**。在过滤框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

5. 点击**创建**。包含您的密钥的 JSON 文件将下载到您的本地环境中。

6. 将您的服务帐号密钥的路径输入到下面的单元格中作为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 关于服务帐号和权限的注意事项

**默认情况下不需要配置**，如果遇到任何与权限有关的问题，请确保上述服务帐号具有所需的角色：

|服务帐号电子邮件|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine默认服务帐号|Dataflow管理员、Dataflow工作人员、存储管理员、BigQuery管理员、Vertex AI用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI平台服务代理|Vertex AI服务代理|


1. 进入https://console.cloud.google.com/iam-admin/iam。
2. 勾选“包括谷歌提供的角色授予”复选框。
3. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请授予这两个服务帐号“BigQuery数据查看器”角色。
- 对于CSV数据源，请授予这两个服务帐号“存储对象查看器”角色。

### 创建一个云存储存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

在下面设置您的云存储存储桶的名称，在本教程中，您将使用它来上传监控服务的`输入模式`。

存储桶的名称在所有Google Cloud项目中必须是全局唯一的，包括组织外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

在本教程中，您将使用与训练预先训练模型相同的公共BigQuery表中的数据。您将创建一个客户端接口，然后使用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### 设置硬件加速器

您可以为预测设置硬件加速器（例如，GPU），或选择不使用任何（CPU）。硬件加速器可以降低预测请求的延迟响应。在选择硬件加速器时，请考虑与延迟之间的额外成本权衡。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持GPU的容器映像，并分配给虚拟机（VM）实例的GPU数量。例如，要使用一个包含4个 Nvidia Tesla K80 GPU 的 GPU 容器映像分配给每个 VM，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

请参阅[可用的加速器位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，请指定 `(None, None)` 以使用一个运行在 CPU 上的容器镜像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]中的数字

*注意：您也可以使用n2和e2机器类型用于训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 介绍 Vertex AI 模型监控

Vertex AI 模型监控支持 AutoML 表格模型和自定义表格模型。您可以监视传入预测请求中特征的偏移和漂移检测，或者监视特征归因（可解释 AI）在传出预测响应中的偏移和漂移 — 即，归因对输出（预测）的贡献方式的分布。

以下是启用模型监控的基本步骤：

1. 将 `Vertex AI` AutoML 或自定义表格模型部署到 `Vertex AI Endpoint`。
2. 配置模型监控规范。
3. 将模型监控规范上传到 `Vertex AI Endpoint`。
4. 上传或自动生成用于解析的 `输入模式`。
5. 对于特征偏移检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的 `Vertex AI Explainability` 规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控后，采样的传入预测请求将被记录到一个 BigQuery 表中。然后分析记录请求中包含的输入特征值，以便根据特定的间隔进行偏移或漂移检测。您可以设置采样率以监视模型的一部分生产输入，并设置监测间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于 AutoML 表格模型，输入模式会自动生成。对于自定义表格模型，服务将尝试从前 1000 个预测请求中自动推导出输入模式。或者，您可以上传输入模式。

对于偏移检测，监控服务需要训练数据中数值的统计分布基线。对于 AutoML 表格模型，这是自动生成的。对于自定义表格模型，您需要将训练数据上传到服务，并让服务自动推导分布。

对于特征归因偏移和漂移检测，需要为您部署的模型启用 `Vertex AI Explainability`。

了解有关 [Vertex AI 模型监控介绍](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview) 的更多信息。

### 为 `Vertex可解释人工智能` 生成可解释的元数据

如果您想要在特征归因的输出预测（响应）上进行偏斜和漂移检测，您需要执行以下附加步骤：

- 为模型指定可解释性规范。
- 然后在将模型上传为 `Vertex AI模型` 资源时，包含可解释性规范。
- 然后在将模型监控配置规范上传到相应的 `Vertex AI终端` 资源时，包含可解释性目标配置。

作为第一步，您可以使用助手方法 `SavedModelMetadataBuilder()` 为您的模型创建可解释的AI规范。

了解更多关于[可解释人工智能简介](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

params = {"sampled_shapley_attribution": {"path_count": 10}}
explanation_parameters = aiplatform.explain.ExplanationParameters(params)

builder = saved_model_metadata_builder.SavedModelMetadataBuilder(
    model_path=MODEL_ARTIFACT_URI, outputs_to_explain=["churned_probs"]
)
explanation_metadata = builder.get_metadata_protobuf()

### 将模型文件作为 `Vertex AI Model` 资源上传

接下来，您可以使用 `upload()` 方法将预训练的自定义表格模型文件作为 `Vertex AI Model` 资源上传，需要提供以下参数：

- `display_name`：`Model` 资源的人类可读名称。
- `artifact_uri`：模型文件的 Cloud Storage 位置。
- `serving_container_image`：模型部署到 `Vertex AI Endpoint` 资源时要使用的服务容器镜像。
- `explanation_parameters`：配置模型预测解释的参数。
- `explanation_metadata`：描述模型输入和输出以进行解释的元数据。
- `sync`：确定是否等待过程完成或立即返回（异步）。

了解更多关于[将模型导入到 Vertex AI](https://cloud.google.com/vertex-ai/docs/model-registry/import-model)。

In [ ]:
model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=explanation_parameters,
    explanation_metadata=explanation_metadata,
    sync=True,
)

print(model)

### 部署 `Vertex AI Model` 资源到 `Vertex AI Endpoint` 资源

接下来，您可以使用 `deploy()` 方法将您的 `Vertex AI Model` 资源部署到 `Vertex AI Endpoint` 资源，使用以下参数:

- `deploy_model_display`: 部署模型的人类可读名称。
- `machine_type`: 每个 VM 节点实例的机器类型。
- `min_replica_count`: 为自动扩展而规划的最小节点数。
- `max_replica_count`: 为自动扩展而规划的最大节点数。
- `accelerator_type`: 每个规划节点的 GPU 加速器类型（如果有的话）。
- `accelrator_count`: 每个规划节点的 GPU 加速器数量（如果有的话）。

了解更多关于 [使用 Vertex AI 部署模型](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

## 配置监控作业

配置监控作业包括以下规范：

- `alert_config`：要发送监控警报的电子邮件地址。
- `schedule_config`：用于分析预测的时间窗口。
- `logging_sampling_strategy`：抽样预测请求的速率。
- `drift_config`：要监视的特征和漂移阈值。
- `skew_config`：要监视的特征和偏斜阈值。

了解更多关于[监控特征偏差和漂移](https://cloud.google.com/vertex-ai/docs/model-monitoring/using-model-monitoring)。

### 配置警报规范

首先，您可以使用以下设置配置`alerting_config`规范：

- `user_emails`： 要发送警报的一个或多个电子邮件列表。
- `enable_logging`：将检测到的异常数据流发送到 Cloud Logging。默认值为 False。

了解更多关于[为模型监控作业配置警报](https://cloud.google.com/vertex-ai/docs/model-monitoring/using-model-monitoring#monitor-job)的信息。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您可以使用以下设置配置`schedule_config`规范：

- `monitor_interval`: 将模型监控作业调度间隔设置为小时。最小时间间隔为1小时。

*注意:* REST API中指定的单位为秒。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置抽样规范

接下来，您可以使用以下设置配置 `logging_sampling_strategy` 规范：

- `sample_rate`：以百分比形式（介于0和1之间）指定用于随机抽样监控的率。选择的样本将被记录到一个 BigQuery 表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您将使用以下设置配置 `drift_config` 规范：

- `drift_thresholds`：一个键/值对的字典，其中键是要监视的特征输入的输入特征。值是检测阈值。当未指定时，特征的默认漂移阈值为 0.3（30%）。
- `attribute_drift_threshold`：一个键/值对的字典，其中键是要监视的特征属性漂移的输入特征。值是检测阈值。当未指定时，特征的默认漂移阈值为 0.3（30%）。

*注意：* 启用特征输入或特征属性漂移的漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05
ATTRIBUTION_DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

ATTRIBUTION_DRIFT_THRESHOLDS = {
    "country": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=DRIFT_THRESHOLDS,
    attribute_drift_thresholds=ATTRIBUTION_DRIFT_THRESHOLDS,
)

配置倾斜检测规范

接下来，您可以使用以下设置配置`skew_config`规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为BigQuery表。否则，必须将设置`data_format`设置为以下值之一。数据的位置必须是Cloud Storage位置。
  - `csv`： 
  - `jsonl`：
  - `tf-record`：
- `skew_thresholds`：一个键/值对的字典，其中键是监视特征输入倾斜的输入特征。值是检测阈值。当未指定时，特征的默认倾斜阈值为0.3（30%）。
- `attribute_skew_thresholds`：一个键/值对的字典，其中键是监视特征属性倾斜的输入特征。值是检测阈值。当未指定时，特征的默认倾斜阈值为0.3（30%）。
- `target_field`：用于训练数据集的目标标签

*注意：*启用倾斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

ATTRIBUTE_SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI,
    skew_thresholds=SKEW_THRESHOLDS,
    attribute_skew_thresholds=ATTRIBUTE_SKEW_THRESHOLDS,
    target_field=TARGET,
)

### 组装目标规范

最后，您可以使用以下设置将目标规范 `objective_config` 组装起来：

- `skew_detection_config`：（可选）用于偏斜检测配置的规范。
- `drift_detection_config`：（可选）用于漂移检测配置的规范。
- `explanation_config`：（可选）在启用特征归因监控时用于解释的规范。

In [ ]:
explanation_config = model_monitoring.ExplanationConfig()

objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=explanation_config,
)

### 创建监控任务

使用`aiplatform.ModelDeploymentMonitoringJob.create()`方法，使用您的监控规范，创建一个监控任务，具有以下参数：

- `display_name`：监控任务的人类可读名称。
- `project`：项目ID。
- `region`：地区。
- `endpoint`：要启用监控的`Vertex AI Endpoint`的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job.gca_resource)

#### 监控工作的电子邮件通知。

监控作业的电子邮件通知将发送到警报配置中的电子邮件地址，通知模型监控作业现已启用。

内容将如下：

您好，Vertex AI客户，

您收到此邮件是因为您正在使用Vertex AI模型监控服务。
此邮件是告知您，我们已收到您设置漂移或偏斜检测以监测以下预测端点的请求。从现在开始，传入的预测请求将被采样并记录以进行分析。
原始请求和响应将从预测服务中收集，并保存在bq://[您的项目ID].model_deployment_monitoring_[端点ID].serving_predict中。

#### 监控作业状态

当您启动`Vertex AI Model Monitoring`作业时，它将处于`PENDING`状态，直到计算出`输入模式`和`偏斜分布基线`。这个过程是顺序进行的。在这个例子中，当您使用自动生成的`输入模式`时，服务会一直保持在`PENDING`状态，直到发送第1000个预测请求（随后讨论）。

一旦生成了`输入模式`，则将启动批处理作业以从训练数据生成分布基线。同样，服务会保持在`PENDING`状态，直到计算出基线分布。

一旦生成了基线分布，监控作业将进入`OFFLINE`状态。按照每个时间间隔的基准 - 例如，每小时一次，监控作业在分析采样数据时将进入`RUNNING`状态。完成后，它将返回`OFFLINE`状态，等待下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=churn_{UUID}")
job = jobs[0]
print(job.state)

## 初始化解析以自动生成输入模式

在您的 `Endpoint` 接收到1000个预测请求后，建模服务将自动解析并创建 `输入模式`。

### 创建1000个实例数据

在这个示例中，BigQuery训练数据中的前1000个条目被用作前1000个预测请求。

*注意：* 在这种情况下，每个实例都是一个预测请求。换句话说，发送1000个预测请求的单个实例与发送具有1000个实例的单个预测请求相同。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发起初始预测请求

接下来，您可以使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
response = endpoint.predict(instances=instances)

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 输入模式的自动生成

在模型监控服务收到 1000 个预测请求实例后，监控将开始分析这些预测请求，自动生成一个特征输入的 `输入模式`。

### 基准分布的自动生成

在生成了 `输入模式` 后，监控服务会创建一个批处理作业来分析训练数据，以确定基准分布。

In [ ]:
# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(120)

用于监视的BigQuery表的位置

用于记录采样请求的BigQuery表位于：

`<PROJECT_ID>.model_deployment_monitoring_<ENDPOINT_ID>`.serving_predict，

其中<ENDPOINT_ID>是`Vertex AI Endpoint`资源的数字标识符。

In [ ]:
ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

BQ_MON_TABLE = f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"

table = bigquery.TableReference.from_string(BQ_MON_TABLE)
bq_table = bqclient.get_table(table)

print(bq_table)

### 暂停监控作业

您可以分别使用`pause()`和`resume()`方法来暂停和恢复监控作业。

In [ ]:
monitoring_job.pause()
monitoring_job.resume()

列出监控工作

接下来，您可以使用`list()`方法获取所有监控工作的列表。

In [ ]:
monitoring_jobs = aiplatform.ModelDeploymentMonitoringJob.list()
print(monitoring_jobs)

### 通过过滤器列出监控作业

或者，您可以使用“filter”参数列出作业的子集。在这个示例中，您通过监控作业的显示名称来过滤列表。

In [ ]:
monitoring_jobs = aiplatform.ModelDeploymentMonitoringJob.list(
    filter=f"display_name=churn_{UUID}"
)

print(monitoring_jobs[0].gca_resource)

### 删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

*注意:* 当监控任务处于运行状态时，您无法删除它。您必须先暂停该任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

删除已记录的采样数据

In [ ]:
# Delete the monitoring logged data BigQuery dataset

! bq rm -r -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}

### 使用预定义输入模式创建监视作业

接下来，您将创建另一个监视作业。这次您将加载一个预定义的“输入模式”。一旦加载，监视服务将使用这个“输入模式”而不是从前1000个预测实例中自动生成一个。

#### 创建预定义输入模式

预定义的“输入模式”被指定为一个YAML文件。在这个例子中，您将检索用于训练数据的BigQuery模式，包括特征名称和数据类型，以生成YAML规范。预定义的“输入模式”必须加载到云存储位置。

了解更多关于[用于解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

创建监控作业

最后，您可以使用`create()`方法创建监控作业，并提供以下额外参数：

- `analysis_instance_schema_uri`：包含`输入模式`的YAML文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

删除监控工作

您可以使用`delete()`方法来删除监控工作。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署并删除“Vertex AI Endpoint”资源

您可以使用“delete（）”方法删除您的“Vertex AI Endpoint”资源。在删除之前，必须先取消部署到您的“Vertex AI Endpoint”资源的任何模型。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

#### 删除 `Vertex AI Model` 资源

您可以使用 `delete()` 方法删除您的 `Vertex AI Model` 资源。

In [ ]:
model.delete()

清理

要清理此项目中使用的所有Google Cloud资源，可以删除您用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -r -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}